# Key Features of Pydantic
### Basic Field Types:
- Required fields (using ...)
- Optional fields (using Optional or default values)
- Constrained types (constr, conint)
- Email validation using EmailStr
### Field Constraints:
- Minimum/maximum lengths
- Value ranges
- Custom field metadata (description, examples)
### Complex Types:
- Nested models (Address within User)
- Lists and Dictionaries
- Enums for predefined choices
### Validation:
- Built-in validation (type checking, constraints)
- Custom validators using @field_validator decorator
- Comprehensive error messages
### Model Configuration:
- JSON encoding customization
- Field name aliases
- Enum handling
### Advanced Features:
- Date/time handling
- Custom error messages
- Model serialization/deserialization
## When you run this code, it will:
- Create a valid user instance
- Show the JSON representation
- Attempt to create an invalid user
- Display detailed validation errors

In [2]:
from datetime import datetime
from typing import List, Optional, Dict
from pydantic import (
    BaseModel, 
    EmailStr, 
    Field, 
    ValidationError, 
    field_validator,
    constr,
    conint
)
from enum import Enum

# Custom Enum for status
class UserStatus(str, Enum):
    ACTIVE = "active"
    INACTIVE = "inactive"
    PENDING = "pending"

# Nested model for address
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str = Field(..., min_length=5, max_length=10)

# Main user model demonstrating various features
class User(BaseModel):
    # Basic field types with constraints
    id: int = Field(..., gt=0)  # ... means required
    username: constr(min_length=3, max_length=50)  # Constrained string
    email: EmailStr
    age: conint(ge=18, le=120)  # Constrained integer
    
    # Optional fields
    full_name: Optional[str] = None
    is_admin: bool = False
    
    # Nested model
    address: Address
    
    # Complex types
    tags: List[str] = []
    metadata: Dict[str, str] = {}
    
    # Enum field
    status: UserStatus = UserStatus.PENDING
    
    # Field with description and example
    bio: Optional[str] = Field(
        None,
        description="User's biography",
        example="A passionate developer"
    )
    
    # Timestamps
    created_at: datetime
    updated_at: Optional[datetime] = None
    
    # Custom validators
    @field_validator('username')
    @classmethod
    def username_alphanumeric(cls, v):
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v.lower()
    
    @field_validator('tags')
    @classmethod
    def no_duplicate_tags(cls, v):
        if len(v) != len(set(v)):
            raise ValueError('Duplicate tags are not allowed')
        return v
    
    # Model configuration
    class Config:
        populate_by_name = True
        use_enum_values = True
        json_encoders = {
            datetime: lambda v: v.isoformat()
        }

# Example usage and validation
def main():
    try:
        # Create a valid user
        user = User(
            id=1,
            username="john_doe",
            email="john@example.com",
            age=25,
            full_name="John Doe",
            address=Address(
                street="123 Main St",
                city="New York",
                country="USA",
                postal_code="10001"
            ),
            tags=["developer", "python"],
            metadata={"department": "engineering"},
            created_at=datetime.now()
        )
        
        print("\n=== Valid User ===")
        print(user.json(indent=2))
        
        # Demonstrate validation errors
        print("\n=== Validation Errors ===")
        invalid_user = {
            "id": -1,  # Invalid (must be > 0)
            "username": "j",  # Too short
            "email": "invalid-email",  # Invalid email format
            "age": 15,  # Too young
            "address": {
                "street": "123 Main St",
                "city": "New York",
                "country": "USA",
                "postal_code": "1"  # Too short
            },
            "tags": ["python", "python"],  # Duplicate tags
            "created_at": "invalid-date"  # Invalid date format
        }
        
        User(**invalid_user)
        
    except ValidationError as e:
        print("\nValidation error details:")
        for error in e.errors():
            print(f"Field: {error['loc']}")
            print(f"Error: {error['msg']}")
            print("---")

if __name__ == "__main__":
    main()


Validation error details:
Field: ('username',)
Error: Value error, Username must be alphanumeric
---


# Extra Fields Configuration:
- extra = "forbid": Raises an error if extra fields are provided
- extra = "allow": Accepts and stores extra fields, which can be accessed later
- extra = "ignore": Accepts but discards extra fields

In [1]:
from pydantic import BaseModel
from datetime import datetime
import numpy as np  # Custom type example

# Example 1: Demonstrating 'extra' config
class UserStrict(BaseModel):
    name: str
    age: int

    class Config:
        extra = "forbid"  # Will raise error for extra fields

class UserAllowExtra(BaseModel):
    name: str
    age: int

    class Config:
        extra = "allow"  # Will accept extra fields

class UserIgnoreExtra(BaseModel):
    name: str
    age: int

    class Config:
        extra = "ignore"  # Will silently ignore extra fields

# Testing extra fields behavior
try:
    # This will raise an error because extra fields are forbidden
    user_strict = UserStrict(
        name="John",
        age=30,
        extra_field="This will cause error"  # Extra field
    )
except Exception as e:
    print("UserStrict Error:", e)

# This works - extra fields are allowed and stored
user_allow = UserAllowExtra(
    name="John",
    age=30,
    extra_field="This will be stored"
)
print("\nUserAllowExtra:")
print(user_allow.model_dump())  # Shows all fields including extra
print("Can access extra field:", user_allow.extra_field)

# This works - extra fields are ignored
user_ignore = UserIgnoreExtra(
    name="John",
    age=30,
    extra_field="This will be ignored"  # Extra field will be ignored
)
print("\nUserIgnoreExtra:")
print(user_ignore.model_dump())  # Shows only defined fields
try:
    print(user_ignore.extra_field)  # This will raise an error
except AttributeError as e:
    print("Cannot access ignored field:", e)


UserStrict Error: 1 validation error for UserStrict
extra_field
  Extra inputs are not permitted [type=extra_forbidden, input_value='This will cause error', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

UserAllowExtra:
{'name': 'John', 'age': 30, 'extra_field': 'This will be stored'}
Can access extra field: This will be stored

UserIgnoreExtra:
{'name': 'John', 'age': 30}
Cannot access ignored field: 'UserIgnoreExtra' object has no attribute 'extra_field'


# Arbitrary Types Configuration:
- arbitrary_types_allowed = True: Allows use of custom Python classes and third-party types (like NumPy arrays)
- Without this setting, Pydantic will only allow its built-in supported types
- Useful when working with custom classes or scientific computing libraries

In [6]:

# Example 2: Demonstrating arbitrary_types_allowed
class DataPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class ScientificData(BaseModel):
    name: str
    data_point: DataPoint  # Custom class
    array: np.ndarray      # NumPy array

    class Config:
        arbitrary_types_allowed = True  # Allows custom types

# Without arbitrary_types_allowed, this would raise an error
try:
    # Creating instance with custom types
    scientific_data = ScientificData(
        name="Experiment 1",
        data_point=DataPoint(1, 2),
        array=np.array([1, 2, 3])
    )
    print("\nScientificData:")
    print(f"Name: {scientific_data.name}")
    print(f"DataPoint: x={scientific_data.data_point.x}, y={scientific_data.data_point.y}")
    print(f"Array: {scientific_data.array}")

except Exception as e:
    print("ScientificData Error:", e)



ScientificData:
Name: Experiment 1
DataPoint: x=1, y=2
Array: [1 2 3]


In [8]:

# Example 3: What happens without arbitrary_types_allowed
class ScientificDataStrict(BaseModel):
    name: str
    data_point: DataPoint
    array: np.ndarray

    # No arbitrary_types_allowed in Config

try:
    strict_data = ScientificDataStrict(
        name="Experiment 2",
        data_point=DataPoint(1, 2),
        array=np.array([1, 2, 3])
    )
except Exception as e:
    print("\nScientificDataStrict Error:")
    print(e)

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class '__main__.DataPoint'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.9/u/schema-for-unknown-type

# @field_validator and validate_assignment
### Key Differences:
1. @field_validator:
- Runs during model initialization
- Can validate multiple fields together
- Can transform values
- More flexible and powerful
- Can access other field values using the values parameter
- Runs only once during model creation
2. validate_assignment:
- Runs when fields are modified after model creation
- Only validates the field being modified
- Cannot access other field values
- Simpler, focused on single-field validation
- Provides runtime validation
### Main use cases:
1. Use @field_validator when you need to:
- Validate relationships between multiple fields
- Transform input data
- Implement complex validation logic
- Validate during model initialization
2. Use validate_assignment when you need to:
- Ensure field values remain valid after model creation
- Prevent invalid modifications to model instances
- Maintain data integrity during the lifecycle of the model

In [6]:
from pydantic import BaseModel, validator, ValidationError
from typing import List

# Example 1: @field_validator decorator
class UserWithValidator(BaseModel):
    username: str
    email: str
    age: int
    scores: List[int]

    # Validator runs during model initialization
    @field_validator('username')
    @classmethod
    def validate_username(cls, value):
        if len(value) < 3:
            raise ValueError('Username must be at least 3 characters')
        return value.lower()  # Transform the value

    @field_validator('email')
    @classmethod
    def validate_email(cls, value):
        if '@' not in value:
            raise ValueError('Invalid email format')
        return value

    # Validator that depends on multiple fields
    @field_validator('scores')
    @classmethod
    def validate_scores(cls, value: Optional[List[int]], info):
        if 'age' in info.data and info.data['age'] < 18 and max(value) > 90:
            raise ValueError('Underage users cannot have scores above 90')
        return value

# Example 2: validate_assignment config
class UserWithAssignmentValidation(BaseModel):
    username: str
    email: str
    age: int
    scores: List[int]

    @field_validator('username')
    @classmethod
    def validate_username(cls, value):
        if len(value) < 3:
            raise ValueError('Username must be at least 3 characters')
        return value.lower()

    class Config:
        validate_assignment = True  # Enable validation on attribute assignment

def test_validation_differences():
    print("=== Testing @field_validator ===")
    try:
        # Creating instance with invalid username
        user1 = UserWithValidator(
            username="ab",  # Too short
            email="test@example.com",
            age=20,
            scores=[85, 90, 95]
        )
    except ValidationError as e:
        print("Validation during initialization:")
        print(e)

    try:
        # Creating instance with valid data
        user1 = UserWithValidator(
            username="john",
            email="test@example.com",
            age=15,  # Underage
            scores=[85, 95, 100]  # Score > 90 for underage
        )
    except ValidationError as e:
        print("\nValidation with multiple fields:")
        print(e)

    print("\n=== Testing validate_assignment ===")
    
    # Create valid instance
    user2 = UserWithAssignmentValidation(
        username="john",
        email="test@example.com",
        age=20,
        scores=[85, 90, 95]
    )

    # Try changing username after creation
    try:
        user2.username = "ab"  # Too short
        print("Without validate_assignment, this would succeed")
    except ValidationError as e:
        print("Validation during assignment:")
        print(e)

    # Example with validate_assignment = False
    class UserWithoutAssignmentValidation(BaseModel):
        username: str
        email: str

        @field_validator('username')
        @classmethod
        def validate_username(cls, value):
            if len(value) < 3:
                raise ValueError('Username must be at least 3 characters')
            return value.lower()

    user3 = UserWithoutAssignmentValidation(
        username="john",
        email="test@example.com"
    )
    
    # This will succeed because validate_assignment is False
    user3.username = "ab"
    print(f"\nWithout validate_assignment:")
    print(f"Changed username to invalid value: {user3.username}")

if __name__ == "__main__":
    test_validation_differences()


=== Testing @field_validator ===
Validation during initialization:
1 validation error for UserWithValidator
username
  Value error, Username must be at least 3 characters [type=value_error, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

Validation with multiple fields:
1 validation error for UserWithValidator
scores
  Value error, Underage users cannot have scores above 90 [type=value_error, input_value=[85, 95, 100], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

=== Testing validate_assignment ===
Validation during assignment:
1 validation error for UserWithAssignmentValidation
username
  Value error, Username must be at least 3 characters [type=value_error, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

Without validate_assignment:
Changed username to invalid value: ab


In [13]:

# Example 3: Demonstrating complex validation scenarios
class AdvancedUser(BaseModel):
    username: str
    password: str
    confirm_password: str
    age: int
    roles: List[str]

    class Config:
        validate_assignment = True

    # Pre-validation for password
    @field_validator('password')
    @classmethod
    def password_strength(cls, value):
        if len(value) < 8:
            raise ValueError('Password must be at least 8 characters')
        if not any(c.isupper() for c in value):
            raise ValueError('Password must contain an uppercase letter')
        return value

    # Cross-field validation
    @field_validator('confirm_password')
    @classmethod
    def passwords_match(cls, value: str, info):
        if 'password' in info.data and value != info.data['password']:
            raise ValueError('Passwords do not match')
        return value

    # Custom validation with multiple fields
    @field_validator('roles')
    @classmethod
    def validate_roles(cls, value: List[str], info):
        if 'age' in info.data:
            if info.data['age'] < 18 and 'admin' in value:
                raise ValueError('Users under 18 cannot be admins')
        return value

print("\n=== Testing Advanced Validation ===")
try:
    user = AdvancedUser(
        username="john",
        password="weak",  # Too weak
        confirm_password="weak",
        age=16,
        roles=['user', 'admin']  # Underage admin not allowed
    )
except ValidationError as e:
    print("\nAdvanced validation errors:")
    print(e)


=== Testing Advanced Validation ===

Advanced validation errors:
2 validation errors for AdvancedUser
password
  Value error, Password must be at least 8 characters [type=value_error, input_value='weak', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error
roles
  Value error, Users under 18 cannot be admins [type=value_error, input_value=['user', 'admin'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Examples of using annotated validators
- @field_validator, for field validation
- @model_validator(mode='before'), for validating before class instantiation
- @model_validator(mode='after'), for validating after the class instantiation
- info input parameter in @field_validator to access other fields for cross field validation

In [14]:
from pydantic import BaseModel, field_validator, model_validator, ConfigDict
from typing import List, Optional
from datetime import datetime

class OrderItem(BaseModel):
    product_id: int
    quantity: int
    price_per_unit: float
    discount: float = 0.0

    @field_validator('quantity')
    @classmethod  # classmethod decorator is required in v2
    def validate_quantity(cls, v: int) -> int:
        if v <= 0:
            raise ValueError('Quantity must be positive')
        return v

class Order(BaseModel):
    order_id: int
    customer_id: int
    items: List[OrderItem]
    order_date: datetime
    delivery_date: Optional[datetime] = None
    total_amount: float
    discount_code: Optional[str] = None
    discount_percentage: Optional[float] = None
    final_amount: float
    
    # Optional: Configure model settings
    model_config = ConfigDict(
        validate_assignment=True,
        str_strip_whitespace=True
    )
    
    # Using model_validator(mode='before') for pre-validation
    @model_validator(mode='before')
    @classmethod
    def validate_dates(cls, data: dict) -> dict:
        order_date = data.get('order_date')
        delivery_date = data.get('delivery_date')

        if order_date and delivery_date and isinstance(order_date, datetime) and isinstance(delivery_date, datetime):
            if delivery_date < order_date:
                raise ValueError('Delivery date cannot be earlier than order date')
        
        return data

    # Using model_validator(mode='after') for post-validation
    @model_validator(mode='after')
    def validate_amounts(self) -> 'Order':
        # Calculate total amount from items
        calculated_total = sum(
            item.quantity * item.price_per_unit * (1 - item.discount)
            for item in self.items
        )

        # Validate total_amount matches calculated total
        if abs(self.total_amount - calculated_total) > 0.01:
            raise ValueError(
                f'Total amount ({self.total_amount}) does not match '
                f'calculated total from items ({calculated_total})'
            )

        # Apply discount code if present
        if self.discount_code and self.discount_percentage is None:
            raise ValueError('Discount percentage required when discount code is provided')

        # Calculate final amount
        final_amount = calculated_total
        if self.discount_percentage is not None:
            if not 0 <= self.discount_percentage <= 100:
                raise ValueError('Discount percentage must be between 0 and 100')
            final_amount *= (1 - self.discount_percentage / 100)

        # Validate final amount
        if abs(self.final_amount - final_amount) > 0.01:
            raise ValueError(
                f'Final amount ({self.final_amount}) does not match '
                f'calculated amount ({final_amount})'
            )

        return self

    # Using field_validator for specific field validation
    @field_validator('discount_percentage')
    @classmethod
    def validate_discount(cls, value: Optional[float], info) -> Optional[float]:
        # In v2, use info.data to access other fields
        if value is not None and 'items' in info.data:
            items = info.data['items']
            if isinstance(items, list):
                total_item_discounts = sum(
                    getattr(item, 'discount', 0) 
                    for item in items 
                    if isinstance(item, OrderItem)
                )
                if total_item_discounts > 0 and value > 0:
                    raise ValueError(
                        'Cannot apply both item-level and order-level discounts'
                    )
        return value


In [18]:
# Test 1: Valid order
try:
    order = Order(
        order_id=1,
        customer_id=101,
        items=[
            OrderItem(
                product_id=1,
                quantity=2,
                price_per_unit=10.0
            ),
            OrderItem(
                product_id=2,
                quantity=1,
                price_per_unit=20.0
            )
        ],
        order_date=datetime.now(),
        delivery_date=datetime.now(),
        total_amount=40.0,
        discount_code="SUMMER2024",
        discount_percentage=10.0,
        final_amount=36.0
    )
    print("Valid order created successfully:")
    print(order.model_dump())  # model_dump() instead of dict() in v2

except ValueError as e:
    print(f"Test 1 Error: {e}")

Valid order created successfully:
{'order_id': 1, 'customer_id': 101, 'items': [{'product_id': 1, 'quantity': 2, 'price_per_unit': 10.0, 'discount': 0.0}, {'product_id': 2, 'quantity': 1, 'price_per_unit': 20.0, 'discount': 0.0}], 'order_date': datetime.datetime(2024, 10, 16, 12, 54, 11, 979210), 'delivery_date': datetime.datetime(2024, 10, 16, 12, 54, 11, 979210), 'total_amount': 40.0, 'discount_code': 'SUMMER2024', 'discount_percentage': 10.0, 'final_amount': 36.0}


In [16]:
# Test 2: Invalid dates
try:
    order = Order(
        order_id=2,
        customer_id=102,
        items=[
            OrderItem(
                product_id=1,
                quantity=1,
                price_per_unit=10.0
            )
        ],
        order_date=datetime(2024, 1, 15),
        delivery_date=datetime(2024, 1, 14),  # Earlier than order_date
        total_amount=10.0,
        final_amount=10.0
    )
except ValueError as e:
    print(f"\nTest 2 Error (Invalid dates): {e}")


Test 2 Error (Invalid dates): 1 validation error for Order
  Value error, Delivery date cannot be earlier than order date [type=value_error, input_value={'order_id': 2, 'customer...0, 'final_amount': 10.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [17]:
# Test 3: Invalid total amount
try:
    order = Order(
        order_id=3,
        customer_id=103,
        items=[
            OrderItem(
                product_id=1,
                quantity=2,
                price_per_unit=10.0
            )
        ],
        order_date=datetime.now(),
        total_amount=25.0,  # Incorrect total (should be 20.0)
        final_amount=25.0
    )
except ValueError as e:
    print(f"\nTest 3 Error (Invalid total): {e}")


Test 3 Error (Invalid total): 1 validation error for Order
  Value error, Total amount (25.0) does not match calculated total from items (20.0) [type=value_error, input_value={'order_id': 3, 'customer...0, 'final_amount': 25.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [5]:
# Test 4: Invalid discount combination
try:
    order = Order(
        order_id=4,
        customer_id=104,
        items=[
            OrderItem(
                product_id=1,
                quantity=1,
                price_per_unit=10.0,
                discount=0.2  # 20% item discount
            )
        ],
        order_date=datetime.now(),
        total_amount=8.0,
        discount_code="EXTRA10",
        discount_percentage=10.0,  # Cannot combine with item discount
        final_amount=7.2
    )
except ValueError as e:
    print(f"\nTest 4 Error (Invalid discount combination): {e}")


Test 4 Error (Invalid discount combination): 1 validation error for Order
discount_percentage
  Value error, Cannot apply both item-level and order-level discounts [type=value_error, input_value=10.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error
